In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re

In [5]:
url = "https://www.vlr.gg/events/europe"
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

def comp(balise):
    return balise.name == 'a' and balise.find('span', class_='event-item-desc-item-status mod-completed') is not None

completeds = pd.DataFrame(columns=['Match', 'Name', 'Date'])

# Utiliser la fonction personnalisée pour récupérer les balises <a>
balises_a = soup.find_all(comp)

# Afficher les balises <a> trouvées
for balise_a in balises_a:
    texts = balise_a.find("div", class_="event-item-title")
    text = texts.get_text(strip=True)
    date = balise_a.find("div", class_="event-item-desc-item mod-dates").get_text(strip=True).replace('Dates','')
    if "—" in date:
        periode1, periode2 = date.split("—")
        if len(periode2) <= 2:
            mois1, jour1 = [x.strip() for x in periode1.split()]
            jour2 = periode2
            date = f"{jour1} {mois1} - {jour2} {mois1}"
        elif periode2 == "TBD":
            mois1, jour1 = [x.strip() for x in periode1.split()]
            mois2 = periode2
            date = f"{jour1} {mois1} - {mois2}"
        elif len(periode2) > 2:
            mois1, jour1 = [x.strip() for x in periode1.split()]
            mois2, jour2 = [x.strip() for x in periode2.split()]
            date = f"{jour1} {mois1} - {jour2} {mois2}"
        else:
            mois1, jour1 = [x.strip() for x in periode1.split()]
            date = f"{jour1} {mois1}"
    url = balise_a.get('href')
    match = re.search(r"/event/(\d+)", url)
    match = match.group(1)
    completeds = completeds._append({'Match': match, 'Name': text, 'Date': date}, ignore_index=True)

completeds

,Match,Name,Date
0,1957,VALORANT Super Cup,Jan 5
1,1931,ESA Open Fire All Stars 2023,17 Dec - 30 Dec
2,1933,TESFED Turkish Cup - 2023,16 Dec - 24 Dec
3,1920,Mandatory.gg Cup #3,13 Dec - 19 Dec
4,1934,Harbor's Crest #2,Dec 16
...,...,...,...
185,1436,Get On My Level!,16 Dec - 18 Dec
186,1420,St. Petersburg Governor's Cup Valorant - Super...,16 Dec - 18 Dec
187,1402,Golden Goose #5,10 Dec - 11 Dec
188,1726,Project V Division 1 - Split 2: Stage 3,6 Aug - TBD
